# Build Integrated Daily Table (2020–2023) for 12 Reference Stations

Team: Codellera Andina  
Authors:  
- María Fernanda Molina  
- Nataly Sarmiento  
- Isabela Saud

This notebook builds the core data engine for Periodo 3.  
It combines ground-station observations, ERA5-Land reanalysis, and static spatial/EO features into a single daily table (2020–2023) for 12 selected reference stations in Central/Southern Italy.  
The final table is the input for ERA5–station error diagnostics and feature-based modeling in Periodos 3 and 4.

---

## Description of the Script

1. **Load station metadata**  
   Reads the CSV of the 12 selected stations and creates a GeoDataFrame in `EPSG:4326` with `station_id`, name and coordinates.

2. **Load ECAD station time series**  
   Reads `TX_STAID` ECAD files (daily Tmax), cleans missing values, converts tenths of °C to °C, and builds a continuous daily timeline from 2020-01-01 to 2023-12-31 for each station.

3. **Load ERA5-Land datasets**  
   Loads daily Tmax (`t2m` → °C), total precipitation (`tp`), and wind components (`u10`, `v10`), handling both `time` and `valid_time` dimensions, and extracts values at each station using nearest-neighbor.

4. **Compute wind speed and wind regime**  
   Combines `u10` and `v10` to compute wind speed (`WS`) and classifies wind direction into 8 sectors: `N, NE, E, SE, S, SW, W, NW`.

5. **Merge station data with ERA5 meteorology**  
   Joins `T_station` with `T_ERA5`, wind variables (`u10`, `v10`, `WS`, `wind_regime`) and `precip`, and computes the daily temperature error `error = T_ERA5 - T_station`.

6. **Attach spatial / EO static features**  
   Adds precomputed station attributes: `distance_to_sea_km`, `NDVI_mean`, `urban_fraction` and `env_class` (e.g. urbano/rural, costa/interior).

7. **Add season and organize final schema**  
   Assigns seasons (`DJF`, `MAM`, `JJA`, `SON`), orders the columns in a clear logical structure, and sorts rows by `station_id` and `date`.

8. **Save outputs**  
   Saves the final integrated daily table as:  
   - `stations_daily_with_features.parquet`  
   - `stations_daily_with_features.csv`

In [10]:
# ============================================================
# 01_station_ERA5_build_daily_table.ipynb
# Build daily table 2020–2023 for 12 reference stations
# ============================================================

from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

pd.options.display.max_columns = 50

# --------------------------------------------------------------------
# 0. PATHS & CONSTANTS
# --------------------------------------------------------------------

DATA_DIR     = Path(r"D:\Polimi\GenHack 2025\Data")
STATIONS_DIR = DATA_DIR / "ECA_blend_tx"
ERA5_DIR     = DATA_DIR / "derived-era5-land-daily-statistics"
OUTPUT_DIR   = DATA_DIR / "outputs"

SELECTED_STATIONS_CSV  = OUTPUT_DIR / "selected_stations_central_italy.csv"

DAILY_OUTPUT_PARQUET = OUTPUT_DIR / "stations_daily_with_features.parquet"
DAILY_OUTPUT_CSV     = OUTPUT_DIR / "stations_daily_with_features.csv"

START_DATE = "2020-01-01"
END_DATE   = "2023-12-31"

TARGET_CRS = "EPSG:4326"

# --------------------------------------------------------------------
# ERA5 FILE PATTERNS (MATCH YOUR REAL FILENAMES)
# --------------------------------------------------------------------
ERA5_TMAX_PATTERN = str(ERA5_DIR / "*_2m_temperature_daily_maximum.nc")
ERA5_PRCP_PATTERN = str(ERA5_DIR / "*_total_precipitation_daily_mean.nc")
ERA5_U10_PATTERN  = str(ERA5_DIR / "*_10m_u_component_of_wind_daily_mean.nc")
ERA5_V10_PATTERN  = str(ERA5_DIR / "*_10m_v_component_of_wind_daily_mean.nc")

# Internal variable names (we already confirmed these)
ERA5_VAR_TMAX = "t2m"
ERA5_VAR_PRCP = "tp"
ERA5_VAR_U10  = "u10"
ERA5_VAR_V10  = "v10"


# --------------------------------------------------------------------
# 1. HELPER FUNCTIONS
# --------------------------------------------------------------------

def assign_season(dates: pd.Series) -> pd.Series:
    dates = pd.to_datetime(dates)
    m = dates.dt.month
    season = np.select(
        [
            m.isin([12, 1, 2]),
            m.isin([3, 4, 5]),
            m.isin([6, 7, 8]),
            m.isin([9, 10, 11]),
        ],
        ["DJF", "MAM", "JJA", "SON"],
        default=None,
    )
    return pd.Series(season, index=dates.index)


def load_selected_stations(csv_path: Path) -> gpd.GeoDataFrame:
    df = pd.read_csv(csv_path)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326",
    )
    gdf = gdf.to_crs(TARGET_CRS)
    gdf["station_id"] = gdf["station_id"].astype(str)
    return gdf


def load_ecad_station_timeseries(
    station_id: str,
    stations_dir: Path,
    start_date: str,
    end_date: str,
) -> pd.DataFrame:
    pat1 = str(stations_dir / f"TX_STAID{station_id}.txt")
    pat2 = str(stations_dir / f"TX_STAID*{station_id}.txt")
    matches = glob(pat1) or glob(pat2)
    if not matches:
        raise FileNotFoundError(f"No ECAD TX file found for station_id={station_id}")

    file_path = Path(matches[0])

    df = pd.read_csv(
        file_path,
        skiprows=20,
        skipinitialspace=True,
    )

    if not {"DATE", "TX"}.issubset(df.columns):
        raise ValueError(
            f"ECAD file {file_path} does not contain DATE/TX columns. "
            f"Columns: {df.columns.tolist()}"
        )

    df = df[["DATE", "TX"]].copy()
    df["date"] = pd.to_datetime(df["DATE"].astype(str), format="%Y%m%d")
    df["TX"] = df["TX"].replace(-9999, np.nan)
    df["T_station"] = df["TX"] / 10.0  # tenths °C → °C

    mask = (df["date"] >= start_date) & (df["date"] <= end_date)
    df = df.loc[mask, ["date", "T_station"]].copy()

    full_range = pd.date_range(start=start_date, end=end_date, freq="D")
    df = (
        df.set_index("date")
          .reindex(full_range)
          .rename_axis("date")
          .reset_index()
    )

    df["station_id"] = str(station_id)
    return df[["station_id", "date", "T_station"]]


def load_era5_dataset(pattern: str, var_name: str) -> xr.DataArray:
    """
    Open all NetCDF files matching pattern and return DataArray for var_name
    restricted to [START_DATE, END_DATE]. Handles 'valid_time' vs 'time'.
    """
    files = sorted(glob(pattern))
    if not files:
        raise FileNotFoundError(f"No ERA5 files found for pattern: {pattern}")

    ds = xr.open_mfdataset(files, combine="by_coords")
    if var_name not in ds.data_vars:
        raise KeyError(
            f"Variable '{var_name}' not found in dataset.\n"
            f"Available vars: {list(ds.data_vars)}"
        )

    da = ds[var_name]

    # Detect time dimension: some files use 'valid_time', others 'time'
    if "time" in da.dims:
        time_dim = "time"
    elif "valid_time" in da.dims:
        time_dim = "valid_time"
    else:
        raise KeyError(f"No 'time' or 'valid_time' dimension in ERA5 variable {var_name}")

    da = da.sel({time_dim: slice(START_DATE, END_DATE)})

    # Convert temperature from Kelvin → °C if needed
    if var_name == ERA5_VAR_TMAX:
        da = da - 273.15

    # Rename time dimension to 'time' so downstream code is consistent
    da = da.rename({time_dim: "time"})

    return da


def era5_values_for_stations(
    da: xr.DataArray,
    stations_gdf: gpd.GeoDataFrame,
    var_name: str,
) -> pd.DataFrame:
    """
    Extract values of 'da' at each station's lat/lon using nearest neighbor.
    Returns: station_id, date, <var_name>
    """
    rows = []
    for _, row in stations_gdf.iterrows():
        sid = str(row["station_id"])
        lat = row["lat"]
        lon = row["lon"]

        ts = da.sel(latitude=lat, longitude=lon, method="nearest").to_pandas()
        ts = ts.reset_index().rename(columns={"time": "date", da.name: var_name})
        ts["station_id"] = sid
        rows.append(ts)

    out = pd.concat(rows, ignore_index=True)
    out["date"] = pd.to_datetime(out["date"])
    return out[["station_id", "date", var_name]]


def compute_wind_speed_and_regime(df_u: pd.DataFrame, df_v: pd.DataFrame) -> pd.DataFrame:
    """
    Merge u10 and v10 and compute WS and wind_regime.
    """
    df = df_u.merge(df_v, on=["station_id", "date"], how="inner")
    df["WS"] = np.sqrt(df["u10"]**2 + df["v10"]**2)

    def classify(u, v):
        if np.isnan(u) or np.isnan(v):
            return "unknown"
        ang = np.degrees(np.arctan2(u, v)) % 360
        sectors = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]
        idx = int((ang + 22.5) // 45) % 8
        return sectors[idx]

    df["wind_regime"] = [classify(u, v) for u, v in zip(df["u10"], df["v10"])]
    return df


# --------------------------------------------------------------------
# 2. LOAD SELECTED STATIONS
# --------------------------------------------------------------------

stations_gdf = load_selected_stations(SELECTED_STATIONS_CSV)
print("Selected stations IDs:", stations_gdf["station_id"].tolist())
display(stations_gdf.head())


# --------------------------------------------------------------------
# 3. BUILD DAILY STATION TABLE (T_station)
# --------------------------------------------------------------------

all_station_ts = []
for sid in stations_gdf["station_id"].astype(str).tolist():
    print(f"Loading ECAD Tmax for station {sid}...")
    ts = load_ecad_station_timeseries(
        station_id=sid,
        stations_dir=STATIONS_DIR,
        start_date=START_DATE,
        end_date=END_DATE,
    )
    all_station_ts.append(ts)

station_daily = pd.concat(all_station_ts, ignore_index=True)
station_daily["date"] = pd.to_datetime(station_daily["date"])
station_daily["station_id"] = station_daily["station_id"].astype(str)

print("Station_daily shape:", station_daily.shape)
display(station_daily.head())


# --------------------------------------------------------------------
# 4. LOAD ERA5 & EXTRACT AT STATIONS
# --------------------------------------------------------------------

print("Loading ERA5 Tmax...")
era5_tmax_da = load_era5_dataset(ERA5_TMAX_PATTERN, ERA5_VAR_TMAX)
era5_tmax_df = era5_values_for_stations(era5_tmax_da, stations_gdf, var_name="T_ERA5")

print("Loading ERA5 u10...")
era5_u10_da = load_era5_dataset(ERA5_U10_PATTERN, ERA5_VAR_U10)
era5_u10_df = era5_values_for_stations(era5_u10_da, stations_gdf, var_name="u10")

print("Loading ERA5 v10...")
era5_v10_da = load_era5_dataset(ERA5_V10_PATTERN, ERA5_VAR_V10)
era5_v10_df = era5_values_for_stations(era5_v10_da, stations_gdf, var_name="v10")

print("Loading ERA5 precipitation...")
era5_prcp_da = load_era5_dataset(ERA5_PRCP_PATTERN, ERA5_VAR_PRCP)
era5_prcp_df = era5_values_for_stations(era5_prcp_da, stations_gdf, var_name="precip")

# Wind speed + regime
era5_wind_df = compute_wind_speed_and_regime(era5_u10_df, era5_v10_df)


# --------------------------------------------------------------------
# 5. JOIN STATION DATA + ERA5
# --------------------------------------------------------------------

df = station_daily.copy()

df = df.merge(era5_tmax_df, on=["station_id", "date"], how="left")
df = df.merge(
    era5_wind_df[["station_id", "date", "u10", "v10", "WS", "wind_regime"]],
    on=["station_id", "date"],
    how="left",
)
df = df.merge(era5_prcp_df, on=["station_id", "date"], how="left")

df["error"] = df["T_ERA5"] - df["T_station"]

print("After merging ERA5:", df.shape)
display(df.head())


# --------------------------------------------------------------------
# 6. ATTACH SPATIAL / EO FEATURES (STATIC)
# --------------------------------------------------------------------

station_attrs = (
    stations_gdf[
        [
            "station_id",
            "name",
            "lat",
            "lon",
            "distance_to_sea_km",
            "NDVI_mean",
            "urban_fraction",
            "env_class",
        ]
    ]
    .drop_duplicates("station_id")
    .copy()
)
station_attrs["station_id"] = station_attrs["station_id"].astype(str)

df["station_id"] = df["station_id"].astype(str)
df = df.merge(station_attrs, on="station_id", how="left")

print("After adding spatial features:", df.shape)
display(df.head())


# --------------------------------------------------------------------
# 7. ADD SEASON & ORDER COLUMNS
# --------------------------------------------------------------------

df["season"] = assign_season(df["date"])

cols_order = [
    # Station info
    "station_id", "name", "lat", "lon",

    # Spatial / EO features
    "distance_to_sea_km",
    "NDVI_mean",
    "urban_fraction",
    "env_class",

    # Time
    "date", "season",

    # Station & ERA5 meteorology
    "T_station",
    "T_ERA5",
    "error",
    "u10",
    "v10",
    "WS",
    "wind_regime",
    "precip",
]

cols_order_final = [c for c in cols_order if c in df.columns]
df = df[cols_order_final].sort_values(["station_id", "date"]).reset_index(drop=True)

print("Final daily table shape (single table, 2020–2023):", df.shape)
display(df.head())


# --------------------------------------------------------------------
# 8. SAVE OUTPUT (SINGLE TABLE)
# --------------------------------------------------------------------

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
df.to_parquet(DAILY_OUTPUT_PARQUET, index=False)
df.to_csv(DAILY_OUTPUT_CSV, index=False)

print("Saved outputs:")
print(" -", DAILY_OUTPUT_PARQUET)
print(" -", DAILY_OUTPUT_CSV)



Selected stations IDs: ['26066', '25857', '25859', '26023', '25880', '26036', '26063', '16924', '26033', '25996', '26005', '26061']


,station_id,name,lon,lat,coverage_total,good_summers,bias,rmse,corr,bias_summer,rmse_summer,corr_summer,is_valid_station,name_diag,lon_diag,lat_diag,coverage_total_diag,good_summers_diag,bias_diag,rmse_diag,corr_diag,bias_summer_diag,rmse_summer_diag,corr_summer_diag,is_valid_station_diag,id,imd_mean,urban_fraction,era5_cell_id,NDVI_mean,distance_to_sea_km,env_class,geometry
0,26066,SABAUDIA,13.003333,41.363056,1.0,4,-0.803283,1.464672,0.983968,-0.259986,1.235366,0.924511,True,SABAUDIA,13.003333,41.363056,1.0,4,-0.803283,1.464672,0.983968,-0.259986,1.235366,0.924511,True,1,14.131121,0.141311,41.4000_13.0000,0.468913,3.371863,urban_coastal,POINT (13.00333 41.36306)
1,25857,BATTIPAGLIA,14.981389,40.584444,1.0,4,-1.549356,1.917148,0.987355,-1.351046,1.822694,0.934098,True,BATTIPAGLIA,14.981389,40.584444,1.0,4,-1.549356,1.917148,0.987355,-1.351046,1.822694,0.934098,True,1,37.340995,0.373410,40.6000_15.0000,0.000000,7.524308,urban_coastal,POINT (14.98139 40.58444)
2,25859,GROMOLA,14.990000,40.475000,1.0,4,-2.155212,2.471832,0.984848,-2.010543,2.424046,0.911679,True,GROMOLA,14.990000,40.475000,1.0,4,-2.155212,2.471832,0.984848,-2.010543,2.424046,0.911679,True,1,6.607182,0.066072,40.5000_15.0000,0.445631,3.180360,urban_coastal,POINT (14.99 40.475)
3,26023,ROMA_LANCIANI,12.523056,41.920278,1.0,4,-1.772573,2.084570,0.989430,-1.790955,1.978658,0.971192,True,ROMA_LANCIANI,12.523056,41.920278,1.0,4,-1.772573,2.084570,0.989430,-1.790955,1.978658,0.971192,True,1,60.131531,0.601315,41.9000_12.5000,0.075708,28.427494,urban_interior,POINT (12.52306 41.92028)
4,25880,PIGNATORE_MAGGGIORE,14.191944,41.173333,1.0,4,-1.921295,2.188500,0.990605,-1.777460,2.071526,0.962156,True,PIGNATORE_MAGGGIORE,14.191944,41.173333,1.0,4,-1.921295,2.188500,0.990605,-1.777460,2.071526,0.962156,True,1,9.267340,0.092673,41.2000_14.2000,0.466664,26.707072,urban_interior,POINT (14.19194 41.17333)


Loading ECAD Tmax for station 26066...
Loading ECAD Tmax for station 25857...
Loading ECAD Tmax for station 25859...
Loading ECAD Tmax for station 26023...
Loading ECAD Tmax for station 25880...
Loading ECAD Tmax for station 26036...
Loading ECAD Tmax for station 26063...
Loading ECAD Tmax for station 16924...
Loading ECAD Tmax for station 26033...
Loading ECAD Tmax for station 25996...
Loading ECAD Tmax for station 26005...
Loading ECAD Tmax for station 26061...
Station_daily shape: (17532, 3)


,station_id,date,T_station
0,26066,2020-01-01,15.0
1,26066,2020-01-02,16.0
2,26066,2020-01-03,14.4
3,26066,2020-01-04,14.1
4,26066,2020-01-05,16.1


Loading ERA5 Tmax...
Loading ERA5 u10...
Loading ERA5 v10...
Loading ERA5 precipitation...
After merging ERA5: (17532, 10)


...

After adding spatial features: (17532, 17)


,station_id,date,T_station,T_ERA5,u10,v10,WS,wind_regime,precip,error,name,lat,lon,distance_to_sea_km,NDVI_mean,urban_fraction,env_class
0,26066,2020-01-01,15.0,13.339813,NaN,NaN,NaN,NaN,3.223618e-07,-1.660187,SABAUDIA,41.363056,13.003333,3.371863,0.468913,0.141311,urban_coastal
1,26066,2020-01-02,16.0,13.743561,NaN,NaN,NaN,NaN,6.966292e-07,-2.256439,SABAUDIA,41.363056,13.003333,3.371863,0.468913,0.141311,urban_coastal
2,26066,2020-01-03,14.4,13.266693,NaN,NaN,NaN,NaN,1.214631e-06,-1.133307,SABAUDIA,41.363056,13.003333,3.371863,0.468913,0.141311,urban_coastal
3,26066,2020-01-04,14.1,13.373077,NaN,NaN,NaN,NaN,2.327067e-05,-0.726923,SABAUDIA,41.363056,13.003333,3.371863,0.468913,0.141311,urban_coastal
4,26066,2020-01-05,16.1,15.089935,NaN,NaN,NaN,NaN,1.842573e-06,-1.010065,SABAUDIA,41.363056,13.003333,3.371863,0.468913,0.141311,urban_coastal


Final daily table shape (single table, 2020–2023): (17532, 18)


,station_id,name,lat,lon,distance_to_sea_km,NDVI_mean,urban_fraction,env_class,date,season,T_station,T_ERA5,error,u10,v10,WS,wind_regime,precip
0,16924,CASTEL CELLESI,42.585556,12.160556,53.938996,0.751874,0.004585,rural_interior,2020-01-01,DJF,12.7,11.189423,-1.510577,NaN,NaN,NaN,NaN,3.223618e-07
1,16924,CASTEL CELLESI,42.585556,12.160556,53.938996,0.751874,0.004585,rural_interior,2020-01-02,DJF,12.5,10.870514,-1.629486,NaN,NaN,NaN,NaN,6.966292e-07
2,16924,CASTEL CELLESI,42.585556,12.160556,53.938996,0.751874,0.004585,rural_interior,2020-01-03,DJF,11.1,10.747162,-0.352838,NaN,NaN,NaN,NaN,1.486577e-06
3,16924,CASTEL CELLESI,42.585556,12.160556,53.938996,0.751874,0.004585,rural_interior,2020-01-04,DJF,10.7,10.329620,-0.370380,NaN,NaN,NaN,NaN,3.270637e-05
4,16924,CASTEL CELLESI,42.585556,12.160556,53.938996,0.751874,0.004585,rural_interior,2020-01-05,DJF,11.7,11.614899,-0.085101,NaN,NaN,NaN,NaN,2.753406e-06


Saved outputs:
 - D:\Polimi\GenHack 2025\Data\outputs\stations_daily_with_features.parquet
 - D:\Polimi\GenHack 2025\Data\outputs\stations_daily_with_features.csv


# Extra checks

In [7]:
print("u10 df shape:", era5_u10_df.shape)
print("v10 df shape:", era5_v10_df.shape)
print("wind df shape:", era5_wind_df.shape)

print("\n--- u10 head ---")
display(era5_u10_df.head())

print("\n--- v10 head ---")
display(era5_v10_df.head())

print("\n--- wind (u10+v10+WS) head ---")
display(era5_wind_df.head())



u10 df shape: (8760, 3)
v10 df shape: (8760, 3)
wind df shape: (8760, 6)

--- u10 head ---


,station_id,date,u10
0,26066,2022-01-01,0.895141
1,26066,2022-01-02,1.324346
2,26066,2022-01-03,0.623155
3,26066,2022-01-04,-1.953271
4,26066,2022-01-05,-1.036253



--- v10 head ---


,station_id,date,v10
0,26066,2022-01-01,-1.354617
1,26066,2022-01-02,-1.222766
2,26066,2022-01-03,-0.972609
3,26066,2022-01-04,1.109679
4,26066,2022-01-05,2.711152



--- wind (u10+v10+WS) head ---


,station_id,date,u10,v10,WS,wind_regime
0,26066,2022-01-01,0.895141,-1.354617,1.623658,SE
1,26066,2022-01-02,1.324346,-1.222766,1.802512,SE
2,26066,2022-01-03,0.623155,-0.972609,1.155115,SE
3,26066,2022-01-04,-1.953271,1.109679,2.246476,NW
4,26066,2022-01-05,-1.036253,2.711152,2.902441,N


In [8]:
print("station_daily:", station_daily["date"].min(), "→", station_daily["date"].max())
print("u10 dates    :", era5_u10_df["date"].min(), "→", era5_u10_df["date"].max())
print("v10 dates    :", era5_v10_df["date"].min(), "→", era5_v10_df["date"].max())


station_daily: 2020-01-01 00:00:00 → 2023-12-31 00:00:00
u10 dates    : 2022-01-01 00:00:00 → 2023-12-31 00:00:00
v10 dates    : 2022-01-01 00:00:00 → 2023-12-31 00:00:00


In [9]:
sid = "26066"  # or any station_id

df_sid = df[(df["station_id"] == sid) & (df["date"] >= "2022-01-01")].copy()
display(
    df_sid[["date","T_station","T_ERA5","precip","u10","v10","WS","wind_regime"]]
    .head(20)
)


,date,T_station,T_ERA5,precip,u10,v10,WS,wind_regime
16802,2022-01-01,12.3,15.200500,8.127341e-07,0.895141,-1.354617,1.623658,SE
16803,2022-01-02,12.5,12.463715,1.041458e-05,1.324346,-1.222766,1.802512,SE
16804,2022-01-03,13.5,13.241852,5.960881e-05,0.623155,-0.972609,1.155115,SE
16805,2022-01-04,16.3,13.691467,1.828589e-04,-1.953271,1.109679,2.246476,NW
16806,2022-01-05,15.7,15.112885,1.231958e-03,-1.036253,2.711152,2.902441,N
16807,2022-01-06,13.9,11.990387,4.438612e-04,-0.210824,0.255301,0.331097,NW
16808,2022-01-07,15.2,14.205261,4.293411e-05,-0.259994,-1.341400,1.366364,S
16809,2022-01-08,11.7,11.613495,2.001755e-04,1.023457,-2.082873,2.320738,SE
16810,2022-01-09,7.0,8.339691,3.487359e-03,0.273814,-1.489625,1.514581,S
16811,2022-01-10,11.6,10.467316,4.361306e-04,0.855347,-3.123544,3.238540,S
